<a href="https://colab.research.google.com/github/JackHidary/quantumcomputingbook/blob/master/Notes/IonDeviceTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ion Device Class**
This notebook provides an introductiong to making circuits that are compatable with  ion qubit devices in Cirq. IonDevice classes are available starting with release 0.5.0.

In [0]:
# install release containing NeutralAtomDevice and IonDevice classes
!pip install cirq~=0.5.0

## IonDevice
*Disclaimer: As with any proposed architecture for quantum computing, several research groups around the world are working towards a device based on ion qubits and each research group has a different approach. The IonDevice class will not accurately reflect all such devices. The class assumes a fully connected chain of ions with two-qubit gates implemented by an Ising type coupling known as the Molmer-Sorensen gate. The Molmer-Sorensen gate couples ions through the shared motional modes of the ion chain.  The ion motion and internal state decouples at the end of each gate. The class assumes this decoupling is perfect and does not explicitly model the ion motion.  This device is modeled on the ion trap quantum computer being constructed at Duke University as part of the National Science Foundation STAQ project and the participation in Cirq is part of the National Science Foundation EPiQC project's mission to provide open-source tools for mapping algorithms to hardware.*


The IonDevice is described as a lineary array of qubits (LineQubit).  The user must specify the number of qubits in the line and the duration of gates and measurements.  The gate times and measurement times listed here are conservative values in microseconds.

In [0]:
import cirq
import numpy as np
import cirq.ion as ci
from cirq import Simulator
import itertools
import random
### number of qubits 
qubit_num = 5
### define your qubits as line qubits for a linear ion trap
qubit_list = cirq.LineQubit.range(qubit_num)
### make your ion trap device with desired gate times and qubits
us = 1000*cirq.Duration(nanos=1)
ion_device = ci.IonDevice(measurement_duration=100*us,
                        twoq_gates_duration=200*us,
                        oneq_gates_duration=10*us,
                        qubits=qubit_list)

---

### Native Gate Set
The gates supported by the IonDevice class are single qubit rotations about X, Y, and Z  and two qubit Molmer-Sorensen gates that are rotations about XX. 



---


Some examples of gates in Cirq that the device supports are given below.

In [0]:
# Single Qubit Z rotation by Pi/5 radians
ion_device.validate_gate(cirq.Rz(np.pi/5))
# Single Qubit X rotation by Pi/7 radians 
ion_device.validate_gate(cirq.Rx(np.pi/7))
# Molmer-Sorensen gate by Pi/4 
ion_device.validate_gate(cirq.MS(np.pi/4))

A Toffoli is an example of a gates in Cirq that the IonDevice does not support.

In [0]:
#Controlled gate with non-integer exponent (rotation angle must be a multiple of pi)
ion_device.validate_gate(cirq.TOFFOLI)

AttributeError: ignored

###Bernstein-Vazirani Algorithm
As an example of implementing an algorithm on the IonDevice.  We examine the Bernstein-Vazirani Algorithm (BV). 

**Problem** Given a hidden string of $k$ bits $s$ and an oracle that outputs a single bit $y=x\cdot s$ mod 2, find *s*.  For input string $x$ and output bit $b$, $Oracle([x,b])=[x,b\oplus y]$

**Classical Solution** We can determine $s$ bit by bit by sending bit strings with a single 1 ( $x_l = x_{{0l
}}x_{{1l}}x_{{2l}}..x_{{kl}}$ where $x_{{jl}}=\delta_{{jl}}$). Then each $y$ yields one bit of $s$ and after $k$ queries we have $s$.

**Quantum Solution** BV uses phase kick-back to reveal the value of the $s$ in a single oracle call.  The input string starts in $|0\rangle$ and then Hadamards are applied to yield $\sum_x \frac{1}{2^{{k/2}}} |x\rangle$. The output bit starts in $|1\rangle$ and then a Hadamard is applied yielding $|-\rangle =\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$.  When the Oracle is applied, all strings $x$ where $x\cdot s =0$ mod 2 do not receive a phase.  All strings where $x\cdot s =1$ mod 2  receive a phase of -1.  After the oracle, our state on the input register is  $\sum_x \frac{1}{2^{{k/2}}} (-1)^{{x.s}} |x\rangle$.  When we apply Hadamards to the input register, this yields  $|s\rangle$.  BV allows us to find $s$ in a single oracle call.

### Ion Trap Implementation
Oracles are note a native device gate, so we need to construct it.  A simple oracle can be constructed  by applying  CNOT gates from the input register to the output bit where there is a CNOT from bit $l$ in the input register to the output bit if $s_l=1$.  CNOT gates are also not native ion trap gates.  Below we show how to implement this sequence in Cirq using the decomposition tool to translate from CNOTSs and Hadamards to ion trap gates.







First we define the hidden string $s$ .

In [0]:
### length of your hidden string
string_length = qubit_num-1
### generate all possible strings of length string_length, and randomly choose one as your hidden string
all_strings = ["".join(seq) for seq in itertools.product("01", repeat=string_length)]
hidden_string = random.choice(all_strings)

Then we construct the generic circuit.

In [0]:

### make the circuit for BV with clifford gates
circuit = cirq.Circuit()
circuit.append([cirq.X(qubit_list[qubit_num-1])])
for i in range(qubit_num):
    circuit.append([cirq.H(qubit_list[i])])
for i in range(qubit_num-1):
    if hidden_string[i] == '1':
        circuit.append([cirq.CNOT(qubit_list[i], qubit_list[qubit_num-1])])
for i in range(qubit_num - 1):
    circuit.append([cirq.H(qubit_list[i])])
    circuit.append([cirq.measure(qubit_list[i])])

print("Doing Bernstein-Vazirani algorithm with hidden string",
      hidden_string, "\n")
print("Clifford Circuit:\n")
print(circuit, "\n")

Doing Bernstein-Vazirani algorithm with hidden string 1011 

Clifford Circuit:

              ┌──┐
0: ───H────────@─────H───M───────────
               │
1: ───H───H────┼M────────────────────
               │
2: ───H────────┼─────@───H───M───────
               │     │
3: ───H────────┼─────┼───@───H───M───
               │     │   │
4: ───X───H────X─────X───X───────────
              └──┘ 



Convert to ion trap gates using KAK decomposition and circuit identities for CNOT and H.

In [0]:
### convert the clifford circuit into circuit with ion trap native gates
ion_circuit = ion_device.decompose_circuit(circuit)
print(repr(ion_circuit))
print("Iontrap Circuit: \n", ion_circuit, "\n")

cirq.Circuit(moments=[
    cirq.Moment(operations=[
        cirq.X.on(cirq.LineQubit(4)),
        cirq.Rx(np.pi*1.0).on(cirq.LineQubit(0)),
        cirq.Rx(np.pi*1.0).on(cirq.LineQubit(1)),
        cirq.Rx(np.pi*1.0).on(cirq.LineQubit(2)),
        cirq.Rx(np.pi*1.0).on(cirq.LineQubit(3)),
    ]),
    cirq.Moment(operations=[
        cirq.Ry(np.pi*-0.5).on(cirq.LineQubit(0)),
        cirq.Ry(np.pi*-0.5).on(cirq.LineQubit(1)),
        cirq.Ry(np.pi*-0.5).on(cirq.LineQubit(2)),
        cirq.Ry(np.pi*-0.5).on(cirq.LineQubit(3)),
        cirq.Rx(np.pi*1.0).on(cirq.LineQubit(4)),
    ]),
    cirq.Moment(operations=[
        cirq.Ry(np.pi*-0.5).on(cirq.LineQubit(4)),
        cirq.Rx(np.pi*1.0).on(cirq.LineQubit(1)),
        cirq.Rx(np.pi*1.0).on(cirq.LineQubit(2)),
        cirq.Ry(np.pi*0.5).on(cirq.LineQubit(0)),
        cirq.Ry(np.pi*0.5).on(cirq.LineQubit(3)),
    ]),
    cirq.Moment(operations=[
        cirq.Ry(np.pi*-0.5).on(cirq.LineQubit(1)),
        cirq.Ry(np.pi*-0.5).on(cirq.LineQub

We can use cirq.merge_single_qubit_gates_into_phased_x_z(circuit)

In [0]:
### convert the clifford circuit into circuit with ion trap native gates
ion_circuit = ion_device.decompose_circuit(circuit)
optimized_ion_circuit=cirq.merge_single_qubit_gates_into_phased_x_z(ion_circuit)
print("Iontrap Circuit: \n", ion_circuit, "\n")

Iontrap Circuit: 
 0: ───X────────────────MS(0.25π)───X^0.5─────────────────────────────M───────────
                       │
1: ────────────────────┼───────────M─────────────────────────────────────────────
                       │
2: ────────────────────┼───────────M─────────────────────────────────────────────
                       │
3: ───X────────────────┼────────────────────MS(0.25π)───X^0.5────────────────M───
                       │                    │
4: ───Y^-0.5───────────MS(0.25π)───X^-0.5───MS(0.25π)───X^-0.5─────────────────── 



We can compare then simulate both circuits

In [0]:
### run the ion trap circuit
simulator = Simulator()
clifford_result = simulator.run(circuit)
result = simulator.run(ion_circuit)

measurement_results = ''
for i in range(qubit_num-1):
    if result.measurements[str(i)][0][0]:
        measurement_results += '1'
    else:
        measurement_results += '0'

print("Hidden string is:", hidden_string)
print("Measurement results are:", measurement_results)
print("Found answer using Bernstein-Vazirani:", hidden_string == measurement_results)


Hidden string is: 1001
Measurement results are: 1001
Found answer using Bernstein-Vazirani: True
